# Zurich Instruments LabOne Python API Example

## Integration Weights Measurement

Measures the integration weights for a qubit readout assuming 8 qubits and using gaussian flat
top pulses.

Requirements:
* LabOne Version >= 22.08
* Instruments:
        1 x SHFQA or SHFQC Instrument
* Loopback configuration between input and output of channel 0

-----

In [ ]:
import zhinst
import zhinst.utils.shfqa as shfqa_utils

### 1. Connect the device

Set up the connection with the device. Firstly create an API session with a Data Server. Then the Data Server connects to the device.

In [ ]:
device_id = "dev2345" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection
#interface = "USB" # For USB connection

server_host = "localhost"
server_port = 8004
api_level = 6 # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)

# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

### 2. Configure the SHFQA channel and the scope

Define the parameters of the experiment.

In [ ]:
channel_index = 0
num_qubits = 8
readout_duration = 600e-9
num_segments = 2
num_averages = 50
num_measurements = num_segments * num_averages
scope_channel = 0

Configure and enable the SHFQA channel.

In [ ]:
# Configure the channel
shfqa_utils.configure_channel(
    daq,
    device_id,
    channel_index,
    center_frequency=5e9,
    input_range=0,
    output_range=-5,
    mode="readout",
)

# Enable the channel
daq.set(
    [
        (f"/{device_id}/qachannels/{channel_index}/input/on", 1),
        (f"/{device_id}/qachannels/{channel_index}/output/on", 1),
    ]
)

Configure the scope.

In [ ]:
shfqa_utils.configure_scope(
    daq,
    device_id,
    input_select={scope_channel: f"channel{channel_index}_signal_input"},
    num_samples=int(readout_duration * shfqa_utils.SHFQA_SAMPLING_FREQUENCY),
    trigger_input=f"channel{channel_index}_sequencer_monitor0",
    num_segments=num_segments,
    num_averages=num_averages,
    # compensation for the delay between generator output and input of the integration unit
    trigger_delay=200e-9,
)

### 3. Generate and upload the waveforms

Define the waveforms' parameters.

In [ ]:
import numpy as np

# Parameters for the waveform
frequencies = np.linspace(2e6, 32e6, num_qubits)
pulse_duration = 500e-9
rise_fall_time = 10e-9
sampling_rate = shfqa_utils.SHFQA_SAMPLING_FREQUENCY
scaling = 0.9

rise_fall_len = int(rise_fall_time * sampling_rate)
pulse_len = int(pulse_duration * sampling_rate)

std_dev = rise_fall_len // 10

Generate the waveforms.

In [ ]:
from scipy.signal import gaussian

#  Generate flat top gaussian
gauss = gaussian(2 * rise_fall_len, std_dev)
flat_top_gaussian = np.ones(pulse_len)
flat_top_gaussian[0:rise_fall_len] = gauss[0:rise_fall_len]
flat_top_gaussian[-rise_fall_len:] = gauss[-rise_fall_len:]

flat_top_gaussian *= scaling

pulses = {}
time_vec = np.linspace(0, pulse_duration, pulse_len)

# Modulate complex sinusoids with flat top gaussian envelope
for i, f in enumerate(frequencies):
    pulses[i] = flat_top_gaussian * np.exp(2j * np.pi * f * time_vec)


Upload the waveforms.

In [ ]:
shfqa_utils.write_to_waveform_memory(
    daq, device_id, channel_index, waveforms=pulses
)

### 4. Run the experiment and plot the results

Configure the sequencer.

In [ ]:
shfqa_utils.configure_sequencer_triggering(
    daq, device_id, channel_index, aux_trigger="software_trigger0"
)

Run the experiment for each qubit.

> Note:
> 
> Software triggering (`start_continuous_sw_trigger()`) is used for illustration purposes only. Use a real
> trigger source for actual experiments.

In [ ]:
weights = {}
scopes = {}
for i in range(num_qubits):

    # Generate the sequencer program
    seqc_program = f"""
                repeat({num_measurements}) {{
                    waitDigTrigger(1);
                    startQA(QA_GEN_{i}, 0x0, true,  0, 0x0);
                }}
                """
    # Load the sequencer program
    shfqa_utils.load_sequencer_program(
        daq, device_id, channel_index, sequencer_program=seqc_program
    )

    # Start a measurement
    shfqa_utils.enable_scope(daq, device_id, single=1)
    shfqa_utils.enable_sequencer(daq, device_id, channel_index, single=1)
    shfqa_utils.start_continuous_sw_trigger(
        daq, device_id, num_triggers=num_measurements, wait_time=readout_duration
    )

    # Get results
    scope_data, *_ = shfqa_utils.get_scope_data(daq, device_id, timeout=5)
    scopes[i] = scope_data

    # Calculate weights
    split_data = np.split(scope_data[scope_channel], 2)
    weights[i] = np.conj(split_data[1] - split_data[0])

    

weights


Plot the data measured with the scope for weight calculation.

In [ ]:
import matplotlib.pyplot as plt

for i in range(num_qubits): 
    split_data = np.split(scopes[i][scope_channel], 2)
    ground_state_data = split_data[0]
    excited_state_data = split_data[1]

    time_ticks = np.array(range(len(ground_state_data))) / sampling_rate

    fig = plt.figure(1)
    fig.suptitle("Scope measurement for readout weights")

    ax1 = plt.subplot(211)
    ax1.plot(time_ticks, np.real(ground_state_data))
    ax1.plot(time_ticks, np.imag(ground_state_data))
    ax1.set_title("ground state")
    ax1.set_ylabel("voltage [V]")
    plt.setp(ax1.get_xticklabels(), visible=False)
    ax1.xaxis.get_offset_text().set_visible(False)
    ax1.grid()
    plt.legend(["real", "imag"])

    ax2 = plt.subplot(212, sharex=ax1)
    ax2.plot(time_ticks, np.real(excited_state_data))
    ax2.plot(time_ticks, np.imag(excited_state_data))
    ax2.set_title("excited state")
    ax2.set_xlabel("t [s]")
    ax2.set_ylabel("voltage [V]")
    ax2.grid()

    plt.show()

Plot the qubit readout weights.

In [ ]:
import matplotlib.pyplot as plt

for i in range(num_qubits): 
    time_ticks = np.array(range(len(weights[i]))) / sampling_rate
    plt.plot(time_ticks, np.real(weights[i]))
    plt.plot(time_ticks, np.imag(weights[i]))
    plt.title("Qubit readout weights")
    plt.xlabel("t [s]")
    plt.ylabel("weights [V]")
    plt.legend(["real", "imag"])
    plt.grid()
    plt.show()